# Preparar datos con Keras

Keras proporciona herramientas para convertir datos de formato texto a numérico para preparar un corpus que pueda ser ejecutado por los modelos. En este notebook vamos a tratar:
 - Métodos para procesar datos
 - API `Tokenizer` que codifica documentos y realiza el proceso de validación y prueba
 - Los 4 esquemas de codificación de documentos diferentes que ofrece `Tokenizer`

## Dividir palabras con `text_to_word_sequence`

Un primer pasio trabajando con texto es dividirlo en palabras. Las palabras se llaman **tokens** y el proceso de dividir texto en tokens, **tokenización**. Keras proporciona la función `text_to_word_sequence()` que divide el texto en una lista de palabras, realizando las siguientes tareas:
- Dividir palabras por espacios en blanco
- Filtrar la puntuación
- Convertir texto a minúsculas(`lower=True`)

In [2]:
from keras.preprocessing.text import text_to_word_sequence

text = 'Texto de prueba para primera toma de contacto con CountVectorizer para probar'

result = text_to_word_sequence(text)
result

['texto',
 'de',
 'prueba',
 'para',
 'primera',
 'toma',
 'de',
 'contacto',
 'con',
 'countvectorizer',
 'para',
 'probar']

## Codificación con `one_hot`

Keras proporciona la función `one_hot()` que se puede utilizar para tokenizar y codificar un documento de texto en un solo paso.

- El nombre sugiere que creará un One Hot Encoding, que no es el caso.
- La función es un wrapper para la función `hashing_trick()`.
- La función devuelve una versión codificada en enteros del documento.
- El uso de una función hash significa que puede haber colisiones y no a todas las palabras se les asignarán valores enteros únicos.
- `one_hot()` hará que el texto esté en minúsculas, filtrará la puntuación y dividirá las palabras en función de los espacios en blanco.
- Además del texto, se debe especificar el tamaño del vocabulario (palabras totales).
- El tamaño del vocabulario define el espacio hash desde el cual se codifican las palabras.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre [one_hot()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/one_hot)

Primero verificamos el tamaño del vocabulario.

In [3]:
text = 'Texto de prueba para primera toma de contacto con CountVectorizer para probar'

words = set(text_to_word_sequence(text))
print(words)
vocab_size = len(words)
print(vocab_size)

{'probar', 'de', 'para', 'toma', 'countvectorizer', 'con', 'contacto', 'prueba', 'texto', 'primera'}
10


Podemos juntar esto con la función `one_hot()` y codificar las palabras en el documento.
- El tamaño del vocabulario se incrementa en un tercio para minimizar las colisiones al mezclar palabras
- Se imprimer el tamaño del vocabulario como 10.
- El documento codificado se imprime como una matriz de palabras codificadas con números enteros.

In [4]:
from keras.preprocessing.text import one_hot

result = one_hot(text,
                 n=round(vocab_size*1.3))
result

[1, 10, 3, 2, 6, 9, 10, 10, 10, 9, 2, 7]

Por la naturaleza estocástica de las redes neuronales, los resultados específicos pueden variar.

## Codificación hash con `hashing_trick`

- La función `hashing_trick()` que tokeniza y luego codifica el documento con enteros, al igual que la función `one_hot()`
- Proporciona más flexibilidad, lo que permite especificar la función hash como `hash` u otras funciones como la `md5`.

In [10]:
from keras.preprocessing.text import hashing_trick

text = 'Texto de prueba para primera toma de contacto con CountVectorizer para probar'

words = set(text_to_word_sequence(text))
vocab_size = len(words)

results = hashing_trick(text,
                        n=round(vocab_size*1.3),
                        hash_function='md5')

print(results)

10
[1, 2, 7, 10, 10, 3, 2, 6, 2, 2, 10, 9]
12


## API de Tokenizer

Keras proporciona la API `Tokenizer` para preparar texto que se puede ajustar y reutilizar para preparar varios documentos de texto.

`Tokenizer` debe construirse y luego caber en documentos de texto sin formato o documentos de texto codificados con enteros.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)


In [22]:
from keras.preprocessing.text import Tokenizer

docs = ['Artificial artificial intelligence',
        'Applied Intelligence',
        'Information fusion',
        'Artificial Information',
        'Computer Science']

tokenizer = Tokenizer()

tokenizer.fit_on_texts(docs)
tokenizer

`Tokenizer` proporciona 4 atributos:
- **word_counts**: diccionario de palabra y sus recuentos de ocurrencia.
- **document_count**: número de documentos evaluados.
- **word_index**: diccionario de palabras y sus índices únicos.
- **word_docs**: diccionario de palabras y el número de documentos en el que aparecen.

In [13]:
print(tokenizer.word_counts)
print()
print(tokenizer.document_count)
print()
print(tokenizer.word_index)
print()
print(tokenizer.word_docs)

OrderedDict([('artificial', 2), ('intelligence', 2), ('applied', 1), ('information', 2), ('fusion', 1), ('computer', 1), ('science', 1)])

5

{'artificial': 1, 'intelligence': 2, 'information': 3, 'applied': 4, 'fusion': 5, 'computer': 6, 'science': 7}

defaultdict(<class 'int'>, {'artificial': 2, 'intelligence': 2, 'applied': 1, 'information': 2, 'fusion': 1, 'computer': 1, 'science': 1})


La función `texts_to_matrix()` de `Tokenizer`:
- Crea un vector por documento provisto por entrada
- La longitud de los vectores es el tamaño total del vocabulario
- Proporciona el argumento `mode` que incluye:
  - **`binary`**: Si cada palabra está presente o no en el documento. Opción por defecto.
  - **`count`**: El conteo de palabras en el documento.
  - **`tfidf`**: Puntuación TF-IDF para cada palabra en el documento.
  - **`freq`**: Frecuencia de cada palabra.

In [23]:
encoded = tokenizer.texts_to_matrix(docs,
                                    mode='count')
encoded

array([[0., 2., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1.]])